In [1]:
import os
import re

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid

In [2]:
from constants import CENSUS_CODES
from constants import CENSUS_25, CENSUS_YEARS

Tabulate number of speakers and percent of speakers (for each language) in each CT.
Join language data with geometry. Tag CT geometry with 2021 municipalities, and filter for set of target municipalities and save.

In [3]:
def join_geo_ct(gdf_ct_gta, df_ct_cen):
    """
    """
    gdf_ct_gta['geosid'] = gdf_ct_gta['geosid'].astype(str)
    df_ct_cen['geosid'] = df_ct_cen['geosid'].astype(str)
    df_ct_cen['geosid'] = df_ct_cen['geosid'].apply(lambda x: x[:-2] + '.00' if x.endswith('.0') else x)

    gdf_full = pd.merge(
        gdf_ct_gta,
        df_ct_cen,
        on='geosid',
        how='left'
    )

    # TODO: Select for relevant municipalities only

    return gdf_full

In [9]:
def get_num_speakers(gdf_full, year):
    """
    """
    gdf_small = gdf_full[['geosid', 'geometry']].copy()

    for l_code, c_codes in CENSUS_CODES[year].items():
        if c_codes:
            col = l_code
            if l_code != 'num_tot':
                col = f'num_{l_code}'

            c_code = c_codes[0]

            gdf_small.loc[:, col] = gdf_full.loc[:, c_code]
    
    # TODO: Add not-English

    return gdf_small

In [12]:
def get_pct_speakers(gdf_num_speakers, year):
    """
    """
    gdf_small = gdf_num_speakers[['geosid', 'geometry']].copy()

    for l_code, c_codes in CENSUS_CODES[year].items():
        if c_codes and (l_code != 'num_tot'):
            col_in = f'num_{l_code}'
            col_out = f'pct_{l_code}'

            gdf_small[col_out] = gdf_num_speakers[col_in] / gdf_num_speakers['num_tot']
    
    gdf_small['num_tot'] = gdf_num_speakers['num_tot']

    # TODO: Add not-English

    return gdf_small

In [13]:
for year in tqdm(CENSUS_YEARS):
    # Load data
    gdf_ct_gta = gpd.read_file(f"../data/geo/{year}_ct/ct_gta_{year}.gpkg")
    df_ct_cen = pd.read_csv(f"../data/census/{year}_ct_wide/census_wide_{year}_ct.csv")

    # Join geometric data 
    gdf_full = join_geo_ct(gdf_ct_gta, df_ct_cen)

    # Compute number speakers per language (ie. select columns)
    gdf_num_speakers = get_num_speakers(gdf_full, year)

    # Compute percent speakers per language (ie. divide by total)
    gdf_pct_speakers = get_pct_speakers(gdf_num_speakers, year)

100%|██████████| 1/1 [00:00<00:00,  1.44it/s]
